https://www.kaggle.com/competitions/boston-housing/overview

In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
df_train = pd.read_csv("../Datasets/Boston/train.csv")
df_test = pd.read_csv("../Datasets/Boston/test.csv")

In [3]:
df_train.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [4]:
df_test.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
1,6,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21
2,8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15
3,9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93
4,10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10


In [5]:
df_train = df_train.drop(columns=["ID"])
df_test_index = df_test["ID"]
df_test = df_test.drop(columns=["ID"])

In [6]:
from sklearn.model_selection import GridSearchCV

Y_train = df_train["medv"]
X_train = df_train.drop(columns=["medv"])

clf = xgb.XGBRFRegressor(
    objective="reg:squarederror", tree_method="hist", device="cuda"
)

param_grid = {
    "colsample_bynode": [0.6, 0.8, 1.0],
    "learning_rate": [0.5, 1],
    "max_depth": [4, 5, 6],
    "subsample": [0.6, 0.8],
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=4,
    verbose=2,
    n_jobs=-1,
)

grid_search.fit(X_train, Y_train)
clf = grid_search.best_estimator_

Fitting 4 folds for each of 36 candidates, totalling 144 fits


In [7]:
X_test = df_test
clf.predict(X_test)

D:\Programming\Python\Jupyter\.venv\Lib\site-packages\xgboost\core.py:729: UserWarning: [06:10:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


array([34.23615  , 23.74952  , 16.117756 , 15.390671 , 17.61584  ,
       19.667377 , 20.545479 , 15.984325 , 16.283632 , 18.810354 ,
       19.527447 , 22.456383 , 17.546227 , 15.462223 , 21.658882 ,
       21.236916 , 22.5429   , 29.565441 , 17.407032 , 23.679321 ,
       22.503916 , 23.490562 , 22.4639   , 21.47727  , 21.131096 ,
       22.519012 , 23.315285 , 22.843504 , 22.76887  , 27.127748 ,
       44.44523  , 44.369736 , 29.674908 , 20.199232 , 17.965847 ,
       20.418242 , 17.82684  , 16.663189 , 19.255247 , 19.323654 ,
       15.944081 , 20.121277 , 20.491459 , 16.264084 , 15.669167 ,
       14.5645   , 14.5238   , 15.885097 , 20.576736 , 20.878765 ,
       17.693214 , 31.747406 , 48.140556 , 20.85748  , 21.190525 ,
       22.437643 , 29.04947  , 48.196815 , 35.43777  , 32.131554 ,
       33.606422 , 46.026688 , 20.065838 , 17.522045 , 18.674524 ,
       20.491314 , 23.227728 , 23.37016  , 17.418083 , 20.950657 ,
       31.143848 , 45.20821  , 37.4975   , 37.672245 , 23.4990

In [8]:
answer = pd.DataFrame(clf.predict(X_test), index=df_test_index, columns=["medv"])
answer.index.name = "ID"

In [9]:
answer.to_csv("sumbission.csv")